# Pre-process our tennis data set so we can feed in the data to ML and DL model

We will do the following in his notebook:

## Reading Data
* data from this dataset is split out by year - we will read

## Clean Data
* rename tourney_id because this is misleading - this is a composite of year and tourney id
* clean string columns - remove special characters and accents
* filter out tournments before 1998 (when Federer went pro)
* filter out non-professional tournaments (ie, challenger, satellites)
* clean out tiebreak scores from scores since we will not be using this as part of our features

## Impute Data
* create tourney year column from original tourney_id
* create tourney id column from original tourney_id
* get match minutes from average minutes for the tournament
* player height from average player height
* draw_size from size of tournament
* player rank from average player rank in the tournament - winner rank will be derived from winners, lower rank will be derived from losers


At the end of this notebook, we are left with around 59k entries. The only missing data we have are match stats which we won't be using for the time being


Match data source: https://github.com/JeffSackmann/tennis_atp

In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline



# Reading Dataset

In [2]:
DATASET_DIR = '../datasets'
START_YEAR = 1998
END_YEAR = 2019
years = np.arange(1998, END_YEAR + 1)

matches_orig = pd.DataFrame()
for year in years:
    matches_orig = matches_orig.append(pd.read_csv(f'{DATASET_DIR}/tennis_atp-master/atp_matches_{year}.csv', parse_dates=["tourney_date"]), ignore_index=True)
    
    
print(len(matches_orig))
# read in our ATP data set and parse tournament dates
# matches_orig = pd.read_csv(f'{DATASET_DIR}/tennis_atp-master/ATP.csv', parse_dates=["tourney_date"])
matches_orig.sample(10).T

66348


,14458,21497,30932,30843,44412,6008,25641,58913,17865,45145
tourney_id,2002-403,2004-520,2007-410,2007-D053,2011-421,1999-419,2005-418,2016-0314,2003-468,2011-352
tourney_name,Miami Masters,Roland Garros,Monte Carlo Masters,Davis Cup G2 R2: PAR vs DOM,Canada Masters,Indianapolis,Washington,Gstaad,Estoril,Paris Masters
surface,Hard,Clay,Clay,Clay,Hard,Hard,Hard,Clay,Clay,Hard
draw_size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_level,M,G,M,D,M,A,A,A,A,M
tourney_date,2002-03-18 00:00:00,2004-05-24 00:00:00,2007-04-15 00:00:00,2007-04-06 00:00:00,2011-08-08 00:00:00,1999-08-16 00:00:00,2005-08-01 00:00:00,2016-07-18 00:00:00,2003-04-07 00:00:00,2011-11-07 00:00:00
match_num,57,14,6,4,52,1,43,279,16,28
winner_id,102563,102905,103292,102887,104386,101703,103484,105806,103852,104338
winner_seed,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winner_entry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [3]:
# make all column names lower case so it's easier to remember
matches = matches_orig
matches.columns = [col.strip().lower() for col in matches.columns]

# these columns don't have much data from our EDA so we can't impute. Let's drop them
# for rank points - these are used to determine a player's ranking in the ATP at the moment of the tournament so it's duplicate of rank - will drop this as well
drop_columns = ["loser_entry", "winner_entry", "loser_seed", "winner_seed", "loser_rank_points", "winner_rank_points"]
matches = matches.drop(drop_columns, axis=1)


# before we filter let's fix some data right now 
# tourney_id is actually a little bit mis-leading - it has the format of {year}-{tourney_id} or {year} {tourney_id}
# let's split this
# print(f'Number of year/tourney that has space before pre-processing {len(matches[matches.tourney_id.str.contains(" ")])}')

matches = matches.rename({"tourney_id": "year_tourney_id"}, axis=1)
# matches["year_tourney_id"] = matches["year_tourney_id"].apply(lambda x: x.replace(" ", "-"))
matches["tourney_year"] = matches.year_tourney_id.apply(lambda x: x.split("-")[0])
matches["tourney_id"] = matches.year_tourney_id.apply(lambda x: x.split("-")[1])

# let's verify what we did
# print(matches[matches.year_tourney_id == "1999 495"][["year_tourney_id", "tourney_year", "tourney_id"]])
# print(f'Number of year/tourney that has space after pre-processing {len(matches[matches.year_tourney_id.str.contains(" ")])}')


# we only care about professional tournaments since we are prediction grand slams so let's filter out non-professional tournaments
# ATP was formed in 1972. Federer turned Pro in 1998, we will only look at matches since January 1998
matches = matches[(~matches.tourney_level.isin(["C", "S", "D"])) & (matches.tourney_date > datetime.datetime(1997, 12, 31))]


### Clean String Columns 

Let's standarize data that is non-numeric by stripping out leading/trailing spaces and converting to lowercase

We will also remove any special characters

In [4]:
# keys for the data are player names and tournament names - these are strings
# we also have come categorical columns - ie loser_hand, winner_hand, surface, tourney_level, 
# let's convert any non-numerical column data into lower case and strip
# we will also remove any special characters and accents

import unicodedata
import re

# first let's print one of the columns
print("Before lowering...")
print(matches[:5].loser_hand)

lower_columns = [col for col, dt in matches.dtypes.items() if dt == np.object]
for col in lower_columns:
    print(f'cleaning col {col}')
    matches[col] = matches[col].str.strip()
    matches[col] = matches[col].str.lower()
    matches[col] = matches[col].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    # we won't do this for score because they should look like 6-3 6-7(7)
    if col not in ["score", "year_tourney_id"]:
        matches[col] = matches[col].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x, flags=re.I | re.A))

# check to make sure we've done this correctly
print("After lowering...")
matches[:5].loser_hand

Before lowering...
0    R
1    R
2    R
3    R
4    R
Name: loser_hand, dtype: object
cleaning col year_tourney_id
cleaning col tourney_name
cleaning col surface
cleaning col tourney_level
cleaning col winner_name
cleaning col winner_hand
cleaning col winner_ioc
cleaning col loser_name
cleaning col loser_hand
cleaning col loser_ioc
cleaning col score
cleaning col round
cleaning col tourney_year
cleaning col tourney_id
After lowering...


0    r
1    r
2    r
3    r
4    r
Name: loser_hand, dtype: object

# Cleaning Scores

We will most likely break score into # of sets won by each user and # of games won by each user as feature

The current score has tiebreak points - since we are not going all the way down to point level, we can strip this out

In [5]:
import re

# first let's get some scores with tiebreak so we can test to make sure we did this correctly
tiebreaks = matches[matches.score.str.contains(r"\)")]
print(tiebreaks["score"].head(10))
tb_indexes = tiebreaks.index
tb_indexes

# strip out tiebreak scores
matches.loc[matches.score.str.contains("\)"), "score"] = matches[matches.score.str.contains("\)")].score.apply(lambda x: re.sub(r'\(\d+\)', '', x))





12    6-3 6-7(5) 6-3
15        7-6(5) 7-5
19    6-7(8) 6-2 6-4
21    4-6 7-6(1) 6-1
24    6-7(3) 6-1 7-5
29     7-6(5) 7-6(4)
30    3-6 6-3 7-6(4)
34    6-7(5) 6-4 6-4
36        6-2 7-6(3)
40        7-6(5) 6-4
Name: score, dtype: object


In [6]:
print(matches.iloc[tb_indexes[:10]]["score"])

# verify if there are any other tiebreak scores left
len(matches[matches.score.str.contains(r"\)")]["score"])


12    6-3 6-7 6-3
15        7-6 7-5
19    6-7 6-2 6-4
21    4-6 7-6 6-1
24    6-7 6-1 7-5
29        7-6 7-6
30    3-6 6-3 7-6
34    6-7 6-4 6-4
36        6-2 7-6
40        7-6 6-4
Name: score, dtype: object


0

In [7]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59764 entries, 0 to 66347
Data columns (total 45 columns):
year_tourney_id    59764 non-null object
tourney_name       59764 non-null object
surface            59764 non-null object
draw_size          0 non-null float64
tourney_level      59764 non-null object
tourney_date       59764 non-null datetime64[ns]
match_num          59764 non-null int64
winner_id          59764 non-null int64
winner_name        59764 non-null object
winner_hand        59764 non-null object
winner_ht          57500 non-null float64
winner_ioc         59764 non-null object
winner_age         59764 non-null float64
loser_id           59764 non-null int64
loser_name         59764 non-null object
loser_hand         59764 non-null object
loser_ht           55821 non-null float64
loser_ioc          59764 non-null object
loser_age          59764 non-null float64
score              59764 non-null object
best_of            59764 non-null int64
round              59764 

# Impute Missing Data

We are missing some matches's minutes (ie, length of the match). We can impute this - since matches length might depend on the tournament (ie, surface) and whether the tournament is best of 3 or 5 - we will use this inforamation to impute by using the mean of match minutes for that tournament


In [8]:
## Some matches are misssing minutes

In [9]:
# we are missing some matches's minutes (ie, length of the match)
# we can impute this - since matches length might depend on the tournament (ie, surface) 
# and whether the tournament is best of 3 or 5 - we will use this inforamation 
# to impute by using the mean of match minutes for that tournament
tids = {id for id in matches[matches.minutes.isnull()].tourney_id}
for tid in tids:
    matches.loc[(matches.minutes.isnull()) & (matches.tourney_id == tid), "minutes"] = \
             matches[(matches.minutes.notnull()) & (matches.tourney_id == tid)].minutes.mean()

In [10]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59764 entries, 0 to 66347
Data columns (total 45 columns):
year_tourney_id    59764 non-null object
tourney_name       59764 non-null object
surface            59764 non-null object
draw_size          0 non-null float64
tourney_level      59764 non-null object
tourney_date       59764 non-null datetime64[ns]
match_num          59764 non-null int64
winner_id          59764 non-null int64
winner_name        59764 non-null object
winner_hand        59764 non-null object
winner_ht          57500 non-null float64
winner_ioc         59764 non-null object
winner_age         59764 non-null float64
loser_id           59764 non-null int64
loser_name         59764 non-null object
loser_hand         59764 non-null object
loser_ht           55821 non-null float64
loser_ioc          59764 non-null object
loser_age          59764 non-null float64
score              59764 non-null object
best_of            59764 non-null int64
round              59764 

In [11]:
matches[matches.minutes.isnull()][["year_tourney_id", "minutes"]]

,year_tourney_id,minutes
3031,1998-604,NaN
3032,1998-604,NaN
3033,1998-604,NaN
3034,1998-604,NaN
3035,1998-604,NaN
3036,1998-604,NaN
3037,1998-604,NaN
3038,1998-604,NaN
3039,1998-604,NaN
3040,1998-604,NaN


Looks like there is still a subset of tournaments where we do not have any results. This is ok - minutes will be used as part of feature engineering. We will just have to skip over these records when we create features later.

Looks like '98 and '99 Grand Slam Cup did not record match minutes. This is a year end tournament: 
https://www.grandslamhistory.com/atp/grand-slam-cup-munich

In [12]:
matches[matches.year_tourney_id.isin(matches[matches.minutes.isnull()].year_tourney_id.tolist())]["tourney_name"].unique()

array(['grand slam cup'], dtype=object)

## Impute Height

We are missing some values for player height. This, we can impute by using the average height

In [13]:
# impute height with mean of players
matches.loc[matches.loser_ht.isnull(), 'loser_ht'] = matches.loser_ht.mean()
matches.loc[matches.winner_ht.isnull(), 'winner_ht'] = matches.winner_ht.mean()

In [14]:
matches.sample(5).T

,6016,29084,57251,14062,65438
year_tourney_id,1999-419,2006-560,2016-0375,2002-505,2018-5014
tourney_name,indianapolis,us open,montpellier,vina del mar,shanghai masters
surface,hard,hard,hard,clay,hard
draw_size,NaN,NaN,NaN,NaN,NaN
tourney_level,a,g,a,a,m
tourney_date,1999-08-16 00:00:00,2006-08-28 00:00:00,2016-02-01 00:00:00,2002-02-11 00:00:00,2018-10-08 00:00:00
match_num,9,10,276,3,265
winner_id,102796,104252,105357,103581,105023
winner_name,magnus norman,florian mayer,john millman,flavio saretta,sam querrey
winner_hand,r,r,r,r,r


In [15]:
# Lastly, let's drop any rows where we don't have scores for the matches
matches = matches.dropna(axis=0, subset=["score", "minutes"])

### Let's see what else we are missing data for

In [16]:
def print_columns_with_missing_data(m: pd.DataFrame):
    print(m.columns[m.isnull().any()].tolist())
    
    
print_columns_with_missing_data(matches)

['draw_size', 'w_ace', 'w_df', 'w_svpt', 'w_1stin', 'w_1stwon', 'w_2ndwon', 'w_svgms', 'w_bpsaved', 'w_bpfaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stin', 'l_1stwon', 'l_2ndwon', 'l_svgms', 'l_bpsaved', 'l_bpfaced', 'winner_rank', 'loser_rank']


## Missing Rank

At first I thought missing rank might mean that the player

When you look closer, I don't recognize all players, but Tim Henman was an English player that was at the top of his game in 1998 and was in the top 10 according to his wikipedia page: https://en.wikipedia.org/wiki/Tim_Henman

So I think we should somehow impute this

In [17]:
matches[(matches.winner_rank.isnull())][["tourney_date", "year_tourney_id", "round", "tourney_name", "winner_name"]].head(10)

,tourney_date,year_tourney_id,round,tourney_name,winner_name
1872,1998-06-22,1998-540,r128,wimbledon,mikael tillstrom
2251,1998-07-20,1998-418,r64,washington,jimmy arias
2351,1998-07-27,1998-423,r32,los angeles,tim henman
2359,1998-07-27,1998-423,r16,los angeles,tim henman
2363,1998-07-27,1998-423,qf,los angeles,tim henman
2365,1998-07-27,1998-423,sf,los angeles,tim henman
2580,1998-08-17,1998-419,r64,indianapolis,wayne black
2600,1998-08-17,1998-419,r32,indianapolis,wayne black
2692,1998-08-24,1998-80,r32,boston,jiri novak
3078,1998-09-28,1998-327,r32,toulouse,lionel barthez


I think a reasonable way to to impute the data for that tournament. We will take the mean of the player rank in that round and impute into our missing ranks

In [18]:
def impute_missing_rank(matches: pd.DataFrame, missing_list: pd.DataFrame, name_col: str) -> pd.DataFrame:
    for index, row in missing_list.iterrows():
        current_round_matches = matches[(matches.year_tourney_id == row.year_tourney_id) & 
                                        (matches["round"] == row["round"]) & (matches[name_col].notnull())]
        if len(current_round_matches) == 0:
            print(f'Unable to find other matches in this round index: {index} year_tourney_id: {str(row["year_tourney_id"])} round: {row["round"]} column: {name_col}')
        else:
#             print(f'len: {len(current_round_matches)} current mean: {int(current_round_matches[name_col].mean())}')
            matches.loc[index, name_col] = int(current_round_matches[name_col].mean())

In [19]:
losers_missing_rank = matches[matches.loser_rank.isnull()][["year_tourney_id", "round", "loser_name"]]
print(f'Missing loser rank before imputing: {len(losers_missing_rank)}')
print(losers_missing_rank.head(5))
impute_missing_rank(matches, losers_missing_rank, "loser_rank")    

losers_missing_rank = matches[matches.loser_rank.isnull()][["year_tourney_id", "round", "loser_name"]]

print(f'Missing loser rank after imputing: {len(losers_missing_rank)}')

Missing loser rank before imputing: 156
     year_tourney_id round        loser_name
1014        1998-336   r32    wing luen wong
1017        1998-336   r32      peter nyborg
1935        1998-540   r64  mikael tillstrom
2195        1998-321   r64  nicolas lapentti
2269        1998-418   r32       jimmy arias
Unable to find other matches in this round index: 2366 year_tourney_id: 1998-423 round: f column: loser_rank
Unable to find other matches in this round index: 3905 year_tourney_id: 1999-495 round: f column: loser_rank
Missing loser rank after imputing: 2


In [20]:
winner_missing_ranks = matches[matches.winner_rank.isnull()][["year_tourney_id", "round", "winner_name"]]
print(f'Missing winner rank before imputing: {len(winner_missing_ranks)}')

impute_missing_rank(matches, winner_missing_ranks, "winner_rank")    

winner_missing_ranks = matches[matches.winner_rank.isnull()][["year_tourney_id", "round", "winner_name"]]
print(f'Missing winner rank after imputing: {len(winner_missing_ranks)}')

Missing winner rank before imputing: 47
Missing winner rank after imputing: 0


## Impute Draw Size

For single elimination tournaments - we can impute this by the first round

In [21]:
# let's look at some some random samples before we impute
rounds = ['r128', 'r64', 'r32', 'r16']
missing_draw_indexes = matches[matches["round"].isin(rounds)].sample(10, random_state = 1).index
print(missing_draw_indexes)
matches.loc[missing_draw_indexes][["tourney_name", "round", "draw_size"]]

Int64Index([50140, 33555, 24851, 62850, 23345, 54444, 12383, 27960, 753,
            34844],
           dtype='int64')


,tourney_name,round,draw_size
50140,hamburg,r64,NaN
33555,acapulco,r32,NaN
24851,roland garros,r32,NaN
62850,doha,r32,NaN
23345,chennai,r16,NaN
54444,sao paulo,r32,NaN
12383,stuttgart,r64,NaN
27960,hamburg masters,r64,NaN
753,miami masters,r128,NaN
34844,wimbledon,r64,NaN


In [22]:
# impute draw size for single elimination tournament
import re

# we are going to impute in reverse order so that we impute with max
for r in rounds:
    size = int(re.sub("r", "", r))
    # impute draw_size for the entire tournament
    tids = np.unique(matches[(matches["round"] == r) & (matches.draw_size.isnull())][["year_tourney_id"]])
    matches.loc[matches["year_tourney_id"].isin(tids), "draw_size"] = size
    
# let's look at the same random samples and see if they look correct
# round should be < then draw_size
matches.loc[missing_draw_indexes][["tourney_name", "round", "draw_size"]]
    

,tourney_name,round,draw_size
50140,hamburg,r64,64.0
33555,acapulco,r32,32.0
24851,roland garros,r32,128.0
62850,doha,r32,32.0
23345,chennai,r16,32.0
54444,sao paulo,r32,32.0
12383,stuttgart,r64,64.0
27960,hamburg masters,r64,64.0
753,miami masters,r128,128.0
34844,wimbledon,r64,128.0


Let's now look at Round Robin tournaments (rr)

Background - these tournaments generally are divided into pools, then winner of the pools go into a single elimination tournament of 4 at the end

In order to calculate this, we will have to look at how many players were in each tournament (winners and losers)

In [23]:
def get_draw_size(matches: pd.DataFrame, yti: str):
    tourney_matches = matches[matches.year_tourney_id == yti]
    pids = np.unique(np.append(tourney_matches.winner_id, tourney_matches.loser_id))
    matches.loc[matches.year_tourney_id == yti, "draw_size"] = len(pids)
    
    
for i in matches[matches["round"] == 'rr']["year_tourney_id"].unique():
    get_draw_size(matches, i)
    
print(matches[matches["round"] == 'rr'].sample(10, random_state=1)[["tourney_name", "round", "draw_size"]])

# check to see if there are any more tournaments without draw_size
print(f'Tournaments without draw_size left: {len(matches[matches.draw_size.isnull()].year_tourney_id)}')

          tourney_name round  draw_size
30544        las vegas    rr       32.0
39122      tour finals    rr        8.0
65671  next gen finals    rr        8.0
30391     buenos aires    rr       32.0
43500       dusseldorf    rr       20.0
43496       dusseldorf    rr       20.0
46663       dusseldorf    rr       19.0
54021      tour finals    rr        9.0
13576      masters cup    rr        8.0
30124     delray beach    rr       32.0
Tournaments without draw_size left: 0


# Done

OK. Looks like we have cleaned up and inputed as much data as we can as of this point

For this round, we will not be using any of the match statistics so we will not drop rows with empty data for these columns.

However, we will drop the two rows for loser_rank and then save to file so we can move on to feature engineering

### We haven't imputed data for missing match stats - this is ok. For our first round, we will not be using match stats as input into our model

In [24]:
print_columns_with_missing_data(matches)

['w_ace', 'w_df', 'w_svpt', 'w_1stin', 'w_1stwon', 'w_2ndwon', 'w_svgms', 'w_bpsaved', 'w_bpfaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stin', 'l_1stwon', 'l_2ndwon', 'l_svgms', 'l_bpsaved', 'l_bpfaced', 'loser_rank']


### Save our data

In [25]:
matches.to_csv(f'{DATASET_DIR}/atp_matches_preprocessed.csv', index=False)